# Keywords
## En este notebook se modifica la funcion clean (que aparece en GraphOfWords.ipynb) para quedarnos unicamente con verbos. Las demas funciones se modifican para estar de acuerdo con esto.  

In [13]:
import networkx as nx
import spacy
import operator
nlp = spacy.load('en')
from nltk.corpus import stopwords
from nltk import sent_tokenize
import re
import string
stop_words = set(stopwords.words('english'))

## funcion que limpia los textos. Ahora nos quedamos solo con los verbos

In [2]:
def clean_verbs(text):
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    sentences=sent_tokenize(text)
    sentences=[nlp(sentence) for sentence in sentences]
    sentences=[[token.lemma_ for token in sentence if token.pos_=='VERB'] for sentence in sentences]
    text=[item for sublist in sentences for item in sublist]
    text=[word for word in text if not word in stop_words]
    table=str.maketrans('', '', string.punctuation)
    text = [w.translate(table) for w in text] ## removemos los simbolos que funcionan como sustantivos, p.e., %
    text = list(filter(lambda a: a != '', text))
    return text


## ejemplo

In [3]:
text='Sequencing the Neanderthal genome (Green et al., 2010, Prüfer et al., 2014), the Denisovan genome (Reich et al., 2010), and several early modern human genomes from Eurasia (Fu et al., 2014, Fu et al., 2015) has confirmed that archaic hominins left their mark in the genomes of modern humans (Plagnol and Wall, 2006, Sankararaman et al., 2014, Vernot and Akey, 2014, Vernot et al., 2016). Present-day individuals in Eurasia inherited ∼2% of their genome from Neanderthals (Green et al., 2010), and individuals from Oceania inherited ∼5% of their genome from Denisovans (Reich et al., 2010). Suggestive evidence indicates that admixture from other unidentified hominin species occurred in Africa (Hammer et al., 2011, Hsieh et al., 2016, Lachance et al., 2012, Plagnol and Wall, 2006, Wall et al., 2009). To understand the functional, phenotypic, and evolutionary consequences of archaic admixture, it is necessary to identify the specific haplotypes and alleles that were inherited from archaic hominin ancestors (Huerta-Sánchez et al., 2014, Juric et al., 2016, Sankararaman et al., 2014, Simonti et al., 2016, Vernot and Akey, 2014). Approaches to identifying introgressed haplotypes include methods that specifically incorporate reference archaic hominin genome sequences and reference-free methods that do not utilize such information. An example of the former category is the method of Sankararaman et al. (2014), which identifies archaic haplotypes by comparing modern human haplotypes to a reference archaic sequence. The latter category of methods include the S∗ statistic (Plagnol and Wall, 2006), which searches for the mutational signature that ancient admixture leaves in the genomes of present-day humans. The S∗ approach is powerful for finding introgressed haplotypes in the absence of an archaic reference genome because it leverages the unusual mutational characteristics of introgressed haplotypes. Because of the long divergence time between Neanderthals and modern humans, Neanderthals carry many alleles that are specific to their lineage. Such alleles are present on introgressed haplotypes but are absent or rare in African genomes. Further, based on the recent timing of admixture, introgressed haplotypes are expected to be maintained without recombination over distances of approximately 50 kb on average (Sankararaman et al., 2012), resulting in high levels of linkage disequilibrium (LD) between Neanderthal-specific alleles in non-African human genomes. In this study, we develop an S∗-like method that has increased power and is suitable for large-scale genome-wide data. We apply the method to large sets of sequenced data from Eurasia and Oceania and identify putative archaic-specific alleles. We examine the rate at which these alleles match the sequenced archaic genomes and the role of the genes containing these alleles, to obtain insights into the history of the admixture events and their impact on modern human genomes.'
clean_verbs(text)

['sequence',
 'confirm',
 'leave',
 'inherit',
 'inherit',
 'indicate',
 'occur',
 'understand',
 'identify',
 'inherit',
 'identify',
 'include',
 'incorporate',
 'utilize',
 'identify',
 'archaic',
 'compare',
 'include',
 'search',
 'find',
 'leverage',
 'carry',
 'base',
 'expect',
 'maintain',
 'result',
 'develop',
 'increase',
 'apply',
 'sequence',
 'identify',
 'examine',
 'match',
 'sequence',
 'contain',
 'obtain']

In [14]:
text=re.sub("[\(\[].*?[\)\]]", "", text)
sentences=sent_tokenize(text)
sentences=[nlp(sentence) for sentence in sentences]
sentences=[[(token.lemma_,token.pos_,token.tag_) for token in sentence] for sentence in sentences]

In [15]:
sentences

[[('sequence', 'VERB', 'VBG'),
  ('the', 'DET', 'DT'),
  ('neanderthal', 'ADJ', 'JJ'),
  ('genome', 'NOUN', 'NN'),
  (',', 'PUNCT', ','),
  ('the', 'DET', 'DT'),
  ('denisovan', 'PROPN', 'NNP'),
  ('genome', 'NOUN', 'NN'),
  (',', 'PUNCT', ','),
  ('and', 'CCONJ', 'CC'),
  ('several', 'ADJ', 'JJ'),
  ('early', 'ADJ', 'JJ'),
  ('modern', 'ADJ', 'JJ'),
  ('human', 'ADJ', 'JJ'),
  ('genome', 'NOUN', 'NNS'),
  ('from', 'ADP', 'IN'),
  ('eurasia', 'PROPN', 'NNP'),
  (' ', 'SPACE', ''),
  ('have', 'VERB', 'VBZ'),
  ('confirm', 'VERB', 'VBN'),
  ('that', 'ADP', 'IN'),
  ('archaic', 'ADJ', 'JJ'),
  ('hominin', 'NOUN', 'NNS'),
  ('leave', 'VERB', 'VBD'),
  ('-PRON-', 'ADJ', 'PRP$'),
  ('mark', 'NOUN', 'NN'),
  ('in', 'ADP', 'IN'),
  ('the', 'DET', 'DT'),
  ('genome', 'NOUN', 'NNS'),
  ('of', 'ADP', 'IN'),
  ('modern', 'ADJ', 'JJ'),
  ('human', 'NOUN', 'NNS'),
  ('.', 'PUNCT', '.')],
 [('present', 'ADJ', 'JJ'),
  ('-', 'PUNCT', 'HYPH'),
  ('day', 'NOUN', 'NN'),
  ('individual', 'NOUN', 'NNS'),
 

## grafo de palabras modificado

In [4]:
# K es el largo de la ventana
# digraph indica el tipo de grafo- True = dirigido, False = no dirigido
def graph_weighted_verbs(text,K,digraph):
    text=clean_verbs(text)
    unique_words=list(set(text))
    if digraph==True: ## grafo dirigido o no dirigido
        G=nx.DiGraph()
    else:
        G=nx.Graph()
    for word in unique_words:
        G.add_node(word)
    for word in unique_words: ## recorremos el texto y encontramos los indices de todas las aparicions de word (index_word)
        index_word=[index for index, value in enumerate(text) if value == word]
        ## ahora buscamos las palabras vecinas en una ventana de largo K (hacia adelante)
        for index in index_word:
            for k in range(1,K+1):
                if index+k in range(len(text)):
                    if G.has_edge(text[index],text[index+k])==False:
                        G.add_edge(text[index],text[index+k],weight=1)
                    else:
                        x=G[text[index]][text[index+k]]['weight']
                        G[text[index]][text[index+k]]['weight']=x+1
    
    return G

In [6]:
graph_weighted_verbs(text,4,True)

## keywords segun pagerank

In [7]:
# K es el largo de la ventana
# number_keywords indica el numero de keywords
# digraph indica si queremos un grafo dirigido (True) o no dirigido (False)
def keywords_pagerank_verbs(text,number_keywords,K,digraph):
    n=len(clean_verbs(text))
    G=graph_weighted_verbs(text,K,digraph)
    keywords=nx.pagerank(G, alpha=0.85, weight='weight')
    if n<number_keywords: ## en el caso de que el texto sea muy corto (incluso menor al numero de keywords)
        number_keywords=n
    return list(list(zip(*sorted(keywords.items(), key=operator.itemgetter(1),reverse=True)))[0][:number_keywords])

In [8]:
keywords_pagerank_verbs(text,10,4,True)

['identify',
 'sequence',
 'obtain',
 'inherit',
 'include',
 'apply',
 'increase',
 'develop',
 'result',
 'maintain']

## (3) keywords segun main core

In [11]:
# K es el largo de la ventana
# digraph es el tipo de grafo - True = grafo dirigido, False = grafo no dirigido
# en esta funcion, no es necesario indicar el numero de keywords. Fijamos un maximo de 10 (esto puede ser un parametro)
def keywords_kcore_verbs(text,K,digraph):
    G=graph_weighted_verbs(text,K,digraph)
    G.remove_edges_from(nx.selfloop_edges(G)) ## evitamos ciclos para evitar que Networkx entregue un error
    main_core_nodes=list(nx.k_core(G).nodes())
    n=len(main_core_nodes)
    if n>10:
        return main_core_nodes[:10]
    else:
        return main_core_nodes

In [12]:
keywords_kcore_verbs(text,4,True)


['incorporate',
 'sequence',
 'include',
 'expect',
 'identify',
 'compare',
 'develop',
 'apply',
 'increase',
 'maintain']